In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import math
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [4]:
dataFrame = pd.read_csv('space_decay.csv')

In [5]:
dataFrame

,CCSDS_OMM_VERS,COMMENT,CREATION_DATE,ORIGINATOR,OBJECT_NAME,OBJECT_ID,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,...,RCS_SIZE,COUNTRY_CODE,LAUNCH_DATE,SITE,DECAY_DATE,FILE,GP_ID,TLE_LINE0,TLE_LINE1,TLE_LINE2
0,2,GENERATED VIA SPACE-TRACK.ORG API,2021-11-01T06:46:11,18 SPCS,ARIANE 42P+ DEB,1992-072J,EARTH,TEME,UTC,SGP4,...,MEDIUM,FR,1992.0,FRGUI,NaN,3195178,188614016,0 ARIANE 42P+ DEB,1 26741U 92072J 21304.94919376 .00000883 0...,2 26741 7.7156 90.2410 6528926 243.1216 38...
1,2,GENERATED VIA SPACE-TRACK.ORG API,2021-11-01T04:58:37,18 SPCS,SL-8 DEB,1979-028C,EARTH,TEME,UTC,SGP4,...,SMALL,CIS,1979.0,PKMTR,NaN,3194950,188593285,0 SL-8 DEB,1 26743U 79028C 21304.68908982 .00000079 0...,2 26743 82.9193 299.1120 0030720 158.9093 201...
2,2,GENERATED VIA SPACE-TRACK.ORG API,2021-11-01T06:26:11,18 SPCS,GSAT 1,2001-015A,EARTH,TEME,UTC,SGP4,...,LARGE,IND,2001.0,SRI,NaN,3195026,188609573,0 GSAT 1,1 26745U 01015A 21305.22411368 -.00000165 0...,2 26745 12.1717 16.5368 0237386 250.1248 146...
3,2,GENERATED VIA SPACE-TRACK.ORG API,2021-10-31T18:07:15,18 SPCS,CZ-4 DEB,1999-057MB,EARTH,TEME,UTC,SGP4,...,SMALL,PRC,1999.0,TSC,NaN,3194431,188556894,0 CZ-4 DEB,1 26754U 99057MB 21304.46625230 .00002265 0...,2 26754 98.4781 8.7205 0060618 37.3771 323...
4,2,GENERATED VIA SPACE-TRACK.ORG API,2021-11-01T04:58:37,18 SPCS,CZ-4 DEB,1999-057MC,EARTH,TEME,UTC,SGP4,...,SMALL,PRC,1999.0,TSC,NaN,3194950,188592541,0 CZ-4 DEB,1 26755U 99057MC 21304.74081807 .00002610 0...,2 26755 98.4232 122.0724 0062255 345.1605 27...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14367,2,GENERATED VIA SPACE-TRACK.ORG API,2021-10-29T18:22:04,18 SPCS,TBA - TO BE ASSIGNED,NaN,EARTH,TEME,UTC,SGP4,...,NaN,NaN,NaN,NaN,NaN,3191411,188414015,0 TBA - TO BE ASSIGNED,1 87970U 21285.56271139 +.00354808 +0...,2 87970 99.0098 214.6827 0011899 219.1613 140...
14368,2,GENERATED VIA SPACE-TRACK.ORG API,2021-10-31T18:06:35,18 SPCS,TBA - TO BE ASSIGNED,NaN,EARTH,TEME,UTC,SGP4,...,NaN,NaN,NaN,NaN,NaN,3194430,188550573,0 TBA - TO BE ASSIGNED,1 89171U 21304.27943186 +.00020950 +0...,2 89171 99.0272 141.1159 0055597 76.8926 283...
14369,2,GENERATED VIA SPACE-TRACK.ORG API,2021-10-31T18:07:15,18 SPCS,TBA - TO BE ASSIGNED,NaN,EARTH,TEME,UTC,SGP4,...,NaN,NaN,NaN,NaN,NaN,3194431,188561362,0 TBA - TO BE ASSIGNED,1 T0007U 21304.15669309 -.00000068 0...,2 T0007 31.9916 262.9339 0566728 162.7824 199...
14370,2,GENERATED VIA SPACE-TRACK.ORG API,2021-10-31T18:07:15,18 SPCS,TBA - TO BE ASSIGNED,NaN,EARTH,TEME,UTC,SGP4,...,NaN,NaN,NaN,NaN,NaN,3194431,188561387,0 TBA - TO BE ASSIGNED,1 T0207U 21304.37554955 .00000690 0...,2 T0207 98.6758 348.3068 0058220 187.0288 173...


In [6]:
dataFrame.describe()

,CCSDS_OMM_VERS,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,...,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,LAUNCH_DATE,DECAY_DATE,FILE,GP_ID
count,14372.0,14372.000000,14372.000000,14372.000000,14372.000000,14372.000000,14372.000000,14372.0,14372.000000,14372.0,...,14372.000000,1.437200e+04,14372.000000,14372.000000,14372.000000,14372.000000,14333.000000,0.0,1.437200e+04,1.437200e+04
mean,2.0,12.463847,0.067168,74.354208,182.353111,165.977199,191.015595,0.0,39558.160381,999.0,...,0.000049,2.307751e-07,10636.185210,223.521557,5721.060178,2795.040245,2004.507570,NaN,3.194186e+06,1.885638e+08
std,0.0,4.509305,0.181547,29.626780,116.717713,104.461866,109.873807,0.0,7619.846133,0.0,...,0.002391,1.485939e-05,9580.627227,446.517041,13016.299008,7943.045264,13.565227,NaN,3.119117e+03,1.393616e+05
min,2.0,0.051783,0.000005,0.001400,0.020000,0.008200,0.004800,0.0,26741.000000,999.0,...,-0.014873,-4.210300e-05,6542.850000,87.783000,183.393000,73.279000,1961.000000,NaN,3.142499e+06,1.860926e+08
25%,2.0,13.452144,0.000725,53.055300,71.987350,77.363475,90.001600,0.0,33965.750000,999.0,...,0.000001,0.000000e+00,6956.246750,96.232750,599.876000,546.115000,1997.000000,NaN,3.194431e+06,1.885665e+08
50%,2.0,14.277052,0.003416,86.369200,189.460250,150.483900,206.957400,0.0,40165.500000,999.0,...,0.000010,0.000000e+00,7177.549000,100.861000,843.686000,702.433500,2002.000000,NaN,3.194950e+06,1.885986e+08
75%,2.0,14.963745,0.013505,98.565125,292.484750,258.328550,284.163200,0.0,45173.500000,999.0,...,0.000032,0.000000e+00,7468.055000,107.046000,1207.919500,831.959750,2018.000000,NaN,3.195064e+06,1.886107e+08
max,2.0,16.404097,0.897218,144.586200,359.967900,359.989900,359.981900,0.0,270288.000000,999.0,...,0.225321,1.654800e-03,304049.635000,27808.429000,372136.402000,224547.015000,2021.000000,NaN,3.195727e+06,1.886250e+08


In [7]:
dataFrame.columns

Index(['CCSDS_OMM_VERS', 'COMMENT', 'CREATION_DATE', 'ORIGINATOR',
       'OBJECT_NAME', 'OBJECT_ID', 'CENTER_NAME', 'REF_FRAME', 'TIME_SYSTEM',
       'MEAN_ELEMENT_THEORY', 'EPOCH', 'MEAN_MOTION', 'ECCENTRICITY',
       'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY',
       'EPHEMERIS_TYPE', 'CLASSIFICATION_TYPE', 'NORAD_CAT_ID',
       'ELEMENT_SET_NO', 'REV_AT_EPOCH', 'BSTAR', 'MEAN_MOTION_DOT',
       'MEAN_MOTION_DDOT', 'SEMIMAJOR_AXIS', 'PERIOD', 'APOAPSIS', 'PERIAPSIS',
       'OBJECT_TYPE', 'RCS_SIZE', 'COUNTRY_CODE', 'LAUNCH_DATE', 'SITE',
       'DECAY_DATE', 'FILE', 'GP_ID', 'TLE_LINE0', 'TLE_LINE1', 'TLE_LINE2'],
      dtype='object')

In [8]:
for item in dataFrame.columns:
    print(item)
    print(dataFrame[item].describe())

CCSDS_OMM_VERS
count    14372.0
mean         2.0
std          0.0
min          2.0
25%          2.0
50%          2.0
75%          2.0
max          2.0
Name: CCSDS_OMM_VERS, dtype: float64
COMMENT
count                                 14372
unique                                    1
top       GENERATED VIA SPACE-TRACK.ORG API
freq                                  14372
Name: COMMENT, dtype: object
CREATION_DATE
count                   14372
unique                    191
top       2021-11-01T04:58:37
freq                     4847
Name: CREATION_DATE, dtype: object
ORIGINATOR
count       14372
unique          1
top       18 SPCS
freq        14372
Name: ORIGINATOR, dtype: object
OBJECT_NAME
count              14372
unique              5372
top       FENGYUN 1C DEB
freq                2669
Name: OBJECT_NAME, dtype: object
OBJECT_ID
count         14333
unique        14333
top       1992-072J
freq              1
Name: OBJECT_ID, dtype: object
CENTER_NAME
count     14372
unique        1
top  

In [9]:
dataFrame["OBJECT_TYPE"].describe()

count      14372
unique         4
top       DEBRIS
freq        8431
Name: OBJECT_TYPE, dtype: object

In [10]:
# dataFrame["OBJECT_TYPE"] = dataFrame["OBJECT_TYPE"].apply(lambda x: 1 if x == "DEBRIS" else 0)

In [11]:
# dataFrame["OBJECT_TYPE"]

In [12]:
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14372 entries, 0 to 14371
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CCSDS_OMM_VERS       14372 non-null  int64  
 1   COMMENT              14372 non-null  object 
 2   CREATION_DATE        14372 non-null  object 
 3   ORIGINATOR           14372 non-null  object 
 4   OBJECT_NAME          14372 non-null  object 
 5   OBJECT_ID            14333 non-null  object 
 6   CENTER_NAME          14372 non-null  object 
 7   REF_FRAME            14372 non-null  object 
 8   TIME_SYSTEM          14372 non-null  object 
 9   MEAN_ELEMENT_THEORY  14372 non-null  object 
 10  EPOCH                14372 non-null  object 
 11  MEAN_MOTION          14372 non-null  float64
 12  ECCENTRICITY         14372 non-null  float64
 13  INCLINATION          14372 non-null  float64
 14  RA_OF_ASC_NODE       14372 non-null  float64
 15  ARG_OF_PERICENTER    14372 non-null 

In [13]:
dataFrame = dataFrame.drop_duplicates(subset=['OBJECT_ID'])

In [14]:
object_columns = list(dataFrame.select_dtypes(include=["object"]).columns)

In [15]:
object_columns

['COMMENT',
 'CREATION_DATE',
 'ORIGINATOR',
 'OBJECT_NAME',
 'OBJECT_ID',
 'CENTER_NAME',
 'REF_FRAME',
 'TIME_SYSTEM',
 'MEAN_ELEMENT_THEORY',
 'EPOCH',
 'CLASSIFICATION_TYPE',
 'OBJECT_TYPE',
 'RCS_SIZE',
 'COUNTRY_CODE',
 'SITE',
 'TLE_LINE0',
 'TLE_LINE1',
 'TLE_LINE2']

In [16]:
dataFrame["MEAN_MOTION_DDOT"].describe()

count    1.433400e+04
mean     2.313869e-07
std      1.487907e-05
min     -4.210300e-05
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.654800e-03
Name: MEAN_MOTION_DDOT, dtype: float64

In [17]:
dataFrame["MEAN_MOTION_DOT"].describe()

count    14334.000000
mean         0.000049
std          0.002394
min         -0.014873
25%          0.000001
50%          0.000010
75%          0.000032
max          0.225321
Name: MEAN_MOTION_DOT, dtype: float64

In [18]:
dataFrame = dataFrame.drop(columns=["CCSDS_OMM_VERS" , "COMMENT" , "CREATION_DATE" , "ORIGINATOR" , "OBJECT_NAME" , "OBJECT_ID" , "REF_FRAME" , "TIME_SYSTEM" , "CENTER_NAME" ,"MEAN_ELEMENT_THEORY" , "COUNTRY_CODE" , "EPOCH" , "EPHEMERIS_TYPE" , "CLASSIFICATION_TYPE" , "RCS_SIZE" , "COUNTRY_CODE" , "SITE" , "TLE_LINE0" , "TLE_LINE1" , "TLE_LINE2"])

In [19]:
dataFrame = dataFrame.drop(columns=["BSTAR" , "MEAN_MOTION_DOT" , "MEAN_MOTION_DDOT" ,"DECAY_DATE" , "FILE" , "GP_ID" , "ELEMENT_SET_NO"])

In [20]:
dataFrame

,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,NORAD_CAT_ID,REV_AT_EPOCH,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,LAUNCH_DATE
0,2.921700,0.652893,7.7156,90.2410,243.1216,38.7796,26741,23588,20668.618,492.864,27784.871,796.095,DEBRIS,1992.0
1,13.754973,0.003072,82.9193,299.1120,158.9093,201.3337,26743,50124,7358.038,104.689,1002.507,957.299,DEBRIS,1979.0
2,1.038224,0.023739,12.1717,16.5368,250.1248,146.2900,26745,7798,41197.852,1386.984,35797.696,33841.738,PAYLOAD,2001.0
3,14.775907,0.006062,98.4781,8.7205,37.3771,323.1632,26754,10204,7015.076,97.456,679.465,594.417,DEBRIS,1999.0
4,14.724482,0.006226,98.4232,122.0724,345.1605,27.6061,26755,9622,7031.400,97.796,697.039,609.491,DEBRIS,1999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14329,1.726826,0.504225,11.6205,188.7505,18.9358,354.6612,49375,119,29347.392,833.900,37766.952,8171.563,DEBRIS,2018.0
14330,1.648809,0.453078,10.8285,227.9659,324.3414,277.0792,49376,104,30265.997,873.358,37600.719,10175.005,DEBRIS,2018.0
14331,1.610644,0.483557,10.7227,225.4025,321.7872,12.3257,49378,99,30742.242,894.053,39229.718,9498.496,DEBRIS,2018.0
14332,15.488596,0.000379,51.6457,28.5788,158.6589,26.9938,49379,30972,6798.198,92.972,422.640,417.486,PAYLOAD,2021.0


In [21]:
mask = dataFrame["OBJECT_TYPE"] == "TBA"

# Drop the rows that match the mask
dataFrame = dataFrame.drop(index=dataFrame[mask].index)

In [22]:
dataFrame

,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,NORAD_CAT_ID,REV_AT_EPOCH,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,LAUNCH_DATE
0,2.921700,0.652893,7.7156,90.2410,243.1216,38.7796,26741,23588,20668.618,492.864,27784.871,796.095,DEBRIS,1992.0
1,13.754973,0.003072,82.9193,299.1120,158.9093,201.3337,26743,50124,7358.038,104.689,1002.507,957.299,DEBRIS,1979.0
2,1.038224,0.023739,12.1717,16.5368,250.1248,146.2900,26745,7798,41197.852,1386.984,35797.696,33841.738,PAYLOAD,2001.0
3,14.775907,0.006062,98.4781,8.7205,37.3771,323.1632,26754,10204,7015.076,97.456,679.465,594.417,DEBRIS,1999.0
4,14.724482,0.006226,98.4232,122.0724,345.1605,27.6061,26755,9622,7031.400,97.796,697.039,609.491,DEBRIS,1999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14328,1.947986,0.486499,11.1000,190.7386,42.3158,346.4724,49373,135,27081.827,739.225,33878.971,7528.413,DEBRIS,2018.0
14329,1.726826,0.504225,11.6205,188.7505,18.9358,354.6612,49375,119,29347.392,833.900,37766.952,8171.563,DEBRIS,2018.0
14330,1.648809,0.453078,10.8285,227.9659,324.3414,277.0792,49376,104,30265.997,873.358,37600.719,10175.005,DEBRIS,2018.0
14331,1.610644,0.483557,10.7227,225.4025,321.7872,12.3257,49378,99,30742.242,894.053,39229.718,9498.496,DEBRIS,2018.0


In [23]:
dataFrame["OBJECT_TYPE"].unique()

array(['DEBRIS', 'PAYLOAD', 'ROCKET BODY'], dtype=object)

In [24]:
dataFrame.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 14125 entries, 0 to 14332
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MEAN_MOTION        14125 non-null  float64
 1   ECCENTRICITY       14125 non-null  float64
 2   INCLINATION        14125 non-null  float64
 3   RA_OF_ASC_NODE     14125 non-null  float64
 4   ARG_OF_PERICENTER  14125 non-null  float64
 5   MEAN_ANOMALY       14125 non-null  float64
 6   NORAD_CAT_ID       14125 non-null  int64  
 7   REV_AT_EPOCH       14125 non-null  int64  
 8   SEMIMAJOR_AXIS     14125 non-null  float64
 9   PERIOD             14125 non-null  float64
 10  APOAPSIS           14125 non-null  float64
 11  PERIAPSIS          14125 non-null  float64
 12  OBJECT_TYPE        14125 non-null  object 
 13  LAUNCH_DATE        14125 non-null  float64
dtypes: float64(11), int64(2), object(1)
memory usage: 1.6+ MB


In [25]:
debrisCount = dataFrame["OBJECT_TYPE"].value_counts()["DEBRIS"]

In [26]:
payloadCount = dataFrame["OBJECT_TYPE"].value_counts()["PAYLOAD"]

In [27]:
rocketBodyCount = dataFrame["OBJECT_TYPE"].value_counts()["ROCKET BODY"]

In [28]:
print("Debris = " , debrisCount)
print("Not debris = " , payloadCount + rocketBodyCount)

print("Total = " , debrisCount+payloadCount+rocketBodyCount)

Debris =  8431
Not debris =  5694
Total =  14125


In [29]:
dataFrame["OBJECT_TYPE"] = dataFrame["OBJECT_TYPE"].apply(lambda x: 1 if x == "DEBRIS" else 0)

In [30]:
debrisTrue = dataFrame["OBJECT_TYPE"].value_counts()[1]

In [31]:
debrisFalse = dataFrame["OBJECT_TYPE"].value_counts()[0]

In [32]:
print("Debris = " , debrisTrue)

print("Not debris = " , debrisFalse)

Debris =  8431
Not debris =  5694


In [33]:
debrisFalse == payloadCount + rocketBodyCount

True

In [34]:
debrisTrue == debrisCount

True

In [35]:
dataFrame

,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,NORAD_CAT_ID,REV_AT_EPOCH,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,LAUNCH_DATE
0,2.921700,0.652893,7.7156,90.2410,243.1216,38.7796,26741,23588,20668.618,492.864,27784.871,796.095,1,1992.0
1,13.754973,0.003072,82.9193,299.1120,158.9093,201.3337,26743,50124,7358.038,104.689,1002.507,957.299,1,1979.0
2,1.038224,0.023739,12.1717,16.5368,250.1248,146.2900,26745,7798,41197.852,1386.984,35797.696,33841.738,0,2001.0
3,14.775907,0.006062,98.4781,8.7205,37.3771,323.1632,26754,10204,7015.076,97.456,679.465,594.417,1,1999.0
4,14.724482,0.006226,98.4232,122.0724,345.1605,27.6061,26755,9622,7031.400,97.796,697.039,609.491,1,1999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14328,1.947986,0.486499,11.1000,190.7386,42.3158,346.4724,49373,135,27081.827,739.225,33878.971,7528.413,1,2018.0
14329,1.726826,0.504225,11.6205,188.7505,18.9358,354.6612,49375,119,29347.392,833.900,37766.952,8171.563,1,2018.0
14330,1.648809,0.453078,10.8285,227.9659,324.3414,277.0792,49376,104,30265.997,873.358,37600.719,10175.005,1,2018.0
14331,1.610644,0.483557,10.7227,225.4025,321.7872,12.3257,49378,99,30742.242,894.053,39229.718,9498.496,1,2018.0


In [36]:
columns = dataFrame.columns

for item in columns:
    print(item)

MEAN_MOTION
ECCENTRICITY
INCLINATION
RA_OF_ASC_NODE
ARG_OF_PERICENTER
MEAN_ANOMALY
NORAD_CAT_ID
REV_AT_EPOCH
SEMIMAJOR_AXIS
PERIOD
APOAPSIS
PERIAPSIS
OBJECT_TYPE
LAUNCH_DATE


In [37]:
for item in columns:
    print(item)
    print(dataFrame[item].describe())
    

MEAN_MOTION
count    14125.000000
mean        12.423639
std          4.534790
min          0.051783
25%         13.442620
50%         14.267312
75%         14.928988
max         16.404097
Name: MEAN_MOTION, dtype: float64
ECCENTRICITY
count    14125.000000
mean         0.068144
std          0.182790
min          0.000005
25%          0.000699
50%          0.003525
75%          0.013719
max          0.897218
Name: ECCENTRICITY, dtype: float64
INCLINATION
count    14125.000000
mean        74.128021
std         29.727391
min          0.001400
25%         53.055200
50%         86.346000
75%         98.582000
max        144.586200
Name: INCLINATION, dtype: float64
RA_OF_ASC_NODE
count    14125.000000
mean       183.373056
std        116.527726
min          0.020000
25%         72.456400
50%        190.919200
75%        293.735100
max        359.967900
Name: RA_OF_ASC_NODE, dtype: float64
ARG_OF_PERICENTER
count    14125.000000
mean       165.756693
std        104.495273
min          0.00820

In [38]:
X = dataFrame.drop(columns=["OBJECT_TYPE" , "LAUNCH_DATE"])
y = dataFrame["OBJECT_TYPE"]

selector = SelectKBest(chi2, k=7).fit(X, y)

selected_indices = selector.get_support(indices=True)

print(selected_indices)

selected_features = X.columns[selected_indices]

print(selected_features)

[ 4  6  7  8  9 10 11]
Index(['ARG_OF_PERICENTER', 'NORAD_CAT_ID', 'REV_AT_EPOCH', 'SEMIMAJOR_AXIS',
       'PERIOD', 'APOAPSIS', 'PERIAPSIS'],
      dtype='object')


In [39]:
dataFrame[selected_features]

,ARG_OF_PERICENTER,NORAD_CAT_ID,REV_AT_EPOCH,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS
0,243.1216,26741,23588,20668.618,492.864,27784.871,796.095
1,158.9093,26743,50124,7358.038,104.689,1002.507,957.299
2,250.1248,26745,7798,41197.852,1386.984,35797.696,33841.738
3,37.3771,26754,10204,7015.076,97.456,679.465,594.417
4,345.1605,26755,9622,7031.400,97.796,697.039,609.491
...,...,...,...,...,...,...,...
14328,42.3158,49373,135,27081.827,739.225,33878.971,7528.413
14329,18.9358,49375,119,29347.392,833.900,37766.952,8171.563
14330,324.3414,49376,104,30265.997,873.358,37600.719,10175.005
14331,321.7872,49378,99,30742.242,894.053,39229.718,9498.496


In [66]:
dataFrame["OBJECT_TYPE"]

0        1
1        1
2        0
3        1
4        1
        ..
14328    1
14329    1
14330    1
14331    1
14332    0
Name: OBJECT_TYPE, Length: 14125, dtype: int64

In [40]:
dataFrame.columns

Index(['MEAN_MOTION', 'ECCENTRICITY', 'INCLINATION', 'RA_OF_ASC_NODE',
       'ARG_OF_PERICENTER', 'MEAN_ANOMALY', 'NORAD_CAT_ID', 'REV_AT_EPOCH',
       'SEMIMAJOR_AXIS', 'PERIOD', 'APOAPSIS', 'PERIAPSIS', 'OBJECT_TYPE',
       'LAUNCH_DATE'],
      dtype='object')

# With feature selection


In [51]:
x= dataFrame.iloc[:, [ 4 , 6 , 7 , 8 , 9 , 10 , 11]].values  
y= dataFrame.iloc[:, 12].values

# Without feature selection

In [52]:
# x= dataFrame.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11]].values  
# y= dataFrame.iloc[:, 12].values

In [53]:
print(x)

print(x.size)

print(x.shape)

[[  243.1216 26741.     23588.     ...   492.864  27784.871    796.095 ]
 [  158.9093 26743.     50124.     ...   104.689   1002.507    957.299 ]
 [  250.1248 26745.      7798.     ...  1386.984  35797.696  33841.738 ]
 ...
 [  324.3414 49376.       104.     ...   873.358  37600.719  10175.005 ]
 [  321.7872 49378.        99.     ...   894.053  39229.718   9498.496 ]
 [  158.6589 49379.     30972.     ...    92.972    422.64     417.486 ]]
98875
(14125, 7)


In [54]:
print(y)

print(y.shape)

print(y.size)

[1 1 0 ... 1 1 0]
(14125,)
14125


In [55]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size= 0.25, random_state=0)  

In [56]:
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

In [57]:
x_train

array([[-1.3932827 , -1.60396654, -0.56598411, ...,  0.60836808,
         1.76165838, -0.27313808],
       [ 0.4590544 , -0.28657071, -1.0049437 , ..., -0.26098898,
        -0.36401891, -0.25783491],
       [ 1.01893928,  0.1785281 , -0.00342615, ..., -0.25455223,
        -0.34672182, -0.25051089],
       ...,
       [ 1.49952202,  0.69525333, -0.61194004, ..., -0.27305041,
        -0.39037517, -0.28255251],
       [-1.33820925,  0.92757226, -0.88477278, ..., -0.2462399 ,
        -0.34460121, -0.20824214],
       [ 1.4759604 , -1.14332358,  0.24258148, ..., -0.25115557,
        -0.33561624, -0.2500487 ]])

In [58]:
x_test

array([[-0.86932932,  0.87287283, -0.81559407, ..., -0.27436332,
        -0.3932776 , -0.2852689 ],
       [ 0.44082164, -1.19955951,  0.30448558, ..., -0.2575614 ,
        -0.35545933, -0.25281001],
       [ 1.49557801, -0.11110231, -1.04827657, ...,  2.48856292,
         2.21652975,  4.01475188],
       ...,
       [ 1.82426062, -0.90301485,  1.15638405, ..., -0.25811377,
        -0.34720436, -0.26950134],
       [-0.86840672, -1.5323656 ,  1.55809625, ..., -0.25838584,
        -0.36542048, -0.24098504],
       [-0.73635102,  1.49346552, -1.10269623, ..., -0.24596165,
        -0.34507723, -0.20593595]])

In [59]:
model = LogisticRegression(C=1.623, class_weight=None, dual=False, fit_intercept=True,  
                   intercept_scaling=1, l1_ratio=None, max_iter=5000,  
                   multi_class='multinomial', n_jobs=None, penalty='l2',  
                   random_state=0, solver='newton-cg', tol=0.0001, verbose=True,  
                   warm_start=False)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy*100)

Accuracy: 80.29445073612685


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


In [70]:
testing = [321.7872	,49378,	99,	30742.242,	894.053,	39229.718,	9498.496]
testing = np.array(testing)
testing = testing.reshape(1, 7)

print(testing)

output = model.predict(testing)

print(output)

[[  321.7872 49378.        99.     30742.242    894.053  39229.718
   9498.496 ]]
[1]
